In [1]:
import pandas as pd
import numpy as np 

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from datetime import datetime
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(1)
tf.set_seed = 42


train_feature = np.load('Data/defog/train_feature.npy')
valid_feature = np.load('Data/defog/valid_feature.npy')
test_feature = np.load('Data/defog/test_feature.npy')
train_label = np.load('Data/defog/train_label.npy')
valid_label = np.load('Data/defog/valid_label.npy')
test_label = np.load('Data/defog/test_label.npy')


2023-05-09 09:00:11.385106: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_feature.shape, valid_feature.shape, train_label.shape, valid_label.shape, test_feature.shape, test_label.shape

((29126467, 3),
 (7281617, 3),
 (29126467, 4),
 (7281617, 4),
 (9102021, 3),
 (9102021, 4))

In [3]:
train_feature, valid_feature, test_feature

(array([[-0.9383897 , -0.63071996,  0.40169254],
        [-0.74626718,  1.15504245,  0.42329343],
        [-1.45901083,  0.82581063,  1.12756431],
        ...,
        [ 0.43678394, -0.47212531, -1.22587108],
        [-0.1803416 ,  1.10619151, -0.7546232 ],
        [ 0.09446408,  0.94949735, -0.89907782]]),
 array([[-1.04810926,  0.17965251,  0.95404327],
        [-0.34367632, -0.82143876, -0.74641297],
        [ 0.12979237,  1.43275645,  0.49608204],
        ...,
        [ 0.4771445 ,  1.08426936, -0.20847458],
        [ 1.1514498 ,  1.15661104, -0.72467404],
        [ 0.94376256, -0.16551092,  2.20844567]]),
 array([[-0.09736766,  1.69497109, -0.51493115],
        [ 0.08952019,  0.3549238 ,  0.62354943],
        [ 4.62867138,  1.84387832, -3.23883751],
        ...,
        [-0.51825131,  0.99459367, -0.27116491],
        [ 0.7047527 , -0.83465243, -1.09991586],
        [ 0.29726989, -0.26692546, -0.38839765]]))

In [4]:
# Building the model and pipeline for Tensorflow 
# Preprocess the data
# Turn our data into TensorFlow Datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_feature, train_label))
train_dataset =  train_dataset.batch(2048).prefetch(tf.data.AUTOTUNE)

valid_dataset = tf.data.Dataset.from_tensor_slices((valid_feature,valid_label))
valid_dataset = valid_dataset.batch(2048).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_feature,test_label))
test_dataset = test_dataset.batch(2048).prefetch(tf.data.AUTOTUNE)

2023-05-09 09:00:16.659897: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-09 09:00:16.857461: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-09 09:00:16.857640: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [5]:
# path = 'Data/defog/'
# train_dataset.save(path+'train_dataset')
# valid_dataset.save(path+'valid_dataset')
# test_dataset.save(path+'test_dataset')

In [6]:
# path = 'Data/defog/'
# train_dataset = tf.data.Dataset.load(path+'train_dataset')
# valid_dataset = tf.data.Dataset.load(path+'valid_dataset')
# test_dataset =  tf.data.Dataset.load(path+'test_dataset')
# test_feature = np.load('Data/defog/test_feature.npy')
# test_label = np.load('Data/defog/test_label.npy')

In [7]:
# Check the data shape
# print(
#     f"Train_feature : {train_feature.shape} {train_feature.dtype} \n" 
#     f"Train_labels : {train_label.shape} {train_label.dtype} \n" 
#     f"Valid_feature : {valid_feature.shape} {valid_feature.dtype} \n" 
#     f"Valid_label : {valid_label.shape} {valid_label.dtype} "
#     ) 

print(f"Train : {train_dataset} \n"
      f"Valid : {valid_dataset} \n" 
      f"Test : {test_dataset}")

Train : <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 4), dtype=tf.int64, name=None))> 
Valid : <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 4), dtype=tf.int64, name=None))> 
Test : <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 4), dtype=tf.int64, name=None))>


In [8]:
# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=5) # if val loss decreases for 3 epochs in a row, stop training

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",  
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=3,
                                                 verbose=1, # print out when learning rate goes down 
                                                 min_lr=1e-7)

In [9]:
# Build the model 
epoch = 10
input_shape = 3

inputs = layers.Input(shape=input_shape, name="input_layer")
x = layers.Dense(32, activation ='relu')(inputs)
x = layers.Dense(64, activation = 'relu')(x)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.1)(x)
# x = layers.Dense(128, activation ='relu')(x)
# x = layers.Dense(512, activation = 'relu')(x)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.25)(x)
# x = layers.Dense(1024, activation ='relu')(x)
# x = layers.Dense(512, activation = 'relu')(x)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(128, activation="relu")(x)
# x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(32, activation="relu")(x)
outputs = layers.Dense(4, activation="softmax",name="output_layer")(x)      
model = tf.keras.Model(inputs, outputs) 

model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

model.summary()



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 3)]               0         
                                                                 
 dense (Dense)               (None, 32)                128       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 output_layer (Dense)        (None, 4)                 132       
                                                                 
Total params: 4,452
Trainable params: 4,452
Non-trainable params: 0
_________________________________________________________________


In [10]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import classification_report

In [11]:
import numpy as np
np.random.seed(40)
from mlflow import MlflowClient
from mlflow.entities import ViewType
from mlflow.models.signature import infer_signature

# !mlflow server --backend-store-uri sqlite:///backend.db

In [12]:
def eval_metrics(classifier, test_features, test_labels):
    
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support
    
    # make prediction
    model_preds_probs   = classifier.predict(test_features)
    model_preds         = tf.argmax(model_preds_probs, axis=1)
    predictions         = tf.argmax(test_labels,axis=1)

    
    # Calculate model accuracy
    # model_accuracy = accuracy_score(test_labels, predictions) * 100
    # Calculate model precision, recall and f1 score using "weighted average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(test_labels, 
                                                                                 predictions, 
                                                                                 average="weighted", 
                                                                                 zero_division= 1)
    
    av_precision    = average_precision_score(test_labels, predictions, zero_division= 1)
    
    model_results = {
        # "accuracy": model_accuracy,
        "precision": model_precision,
        "recall": model_recall,
        "f1": model_f1,
        "av precisoin" : av_precision}
    
    
    
    
    
    # target_names = ['StartHesitation','Turn','Walking', 'All_zero']
    # print("Classification report")
    # print("---------------------","\n")
    # print(classification_report(test_labels, predictions,target_names=target_names),"\n")
    # print("Confusion Matrix")
    # print("---------------------","\n")
    # print(f"{Matrix} \n")

    print("Accuracy Measures")
    print("---------------------","\n")
    # print("Accuracy: ", model_accuracy)
    print("Avarge Precision: ", av_precision)
    print("precision: ", model_precision)
    print("recall : ", model_recall)
    print("f1 : ", model_f1)
    
    return model_results

In [13]:
import mlflow
import mlflow.tensorflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")


tracking URI: 'http://127.0.0.1:5000'


In [14]:
import tensorflow as tf
# how to check tensorflow version
tf.__version__
len(tf.config.list_physical_devices('GPU:0'))>0

False

In [16]:
from sklearn.model_selection import cross_val_score
counter = 1
Model_Name = 'DNN_model'
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(1)
tf.set_seed = 42
np.random.seed(40)

with mlflow.start_run():
    mlflow.tensorflow.autolog()
    
    # fit the model
    # with tf.device('/GPU:0'):
    start = datetime.now()
    history_model = model.fit(train_dataset,
                            batch_size=2048,
                            steps_per_epoch=len(train_dataset),
                            validation_data=valid_dataset,
                            validation_steps=int(len(valid_dataset)),
                            callbacks=[early_stopping, reduce_lr],
                            epochs=epoch) 
    end = datetime.now()
    print(f"The time taken to train the model is {end - start}")
        
    # Evaluate model
    model.evaluate(test_dataset)
    
    # Calculate the metrics
    # model_results = eval_metrics(model, 
    #                              test_features = test_feature,
    #                              test_labels = test_label)
    
    mlflow.log_param("Model"           , Model_Name)
    mlflow.log_param("Dataset" , "Defog")
    # mlflow.log_params(model_results)
    mlflow.tensorflow.autolog()
    
    
    print("________________________________________")

Epoch 1/10


2023-05-09 09:01:53.914102: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [29126467,4]
	 [[{{node Placeholder/_1}}]]
2023-05-09 09:01:56.245449: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-09 09:01:56.273013: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x55cf86aec0d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-09 09:01:56.273040: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-05-09 09:01:56.321769: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash repr

14217/14222 [============================>.] - ETA: 0s - loss: 0.8057 - accuracy: 0.6687

2023-05-09 09:02:37.689640: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [7281617,4]
	 [[{{node Placeholder/_1}}]]


14222/14222 [==============================] - 54s 3ms/step - loss: 0.8057 - accuracy: 0.6687 - val_loss: 0.7512 - val_accuracy: 0.6909 - lr: 0.0010
Epoch 2/10
14222/14222 [==============================] - 48s 3ms/step - loss: 0.7326 - accuracy: 0.6979 - val_loss: 0.7238 - val_accuracy: 0.7007 - lr: 0.0010
Epoch 3/10
14222/14222 [==============================] - 48s 3ms/step - loss: 0.7168 - accuracy: 0.7034 - val_loss: 0.7143 - val_accuracy: 0.7038 - lr: 0.0010
Epoch 4/10
14222/14222 [==============================] - 47s 3ms/step - loss: 0.7091 - accuracy: 0.7060 - val_loss: 0.7086 - val_accuracy: 0.7055 - lr: 0.0010
Epoch 5/10
14222/14222 [==============================] - 48s 3ms/step - loss: 0.7041 - accuracy: 0.7078 - val_loss: 0.7043 - val_accuracy: 0.7068 - lr: 0.0010
Epoch 6/10
14222/14222 [==============================] - 48s 3ms/step - loss: 0.7002 - accuracy: 0.7090 - val_loss: 0.7021 - val_accuracy: 0.7076 - lr: 0.0010
Epoch 7/10
14222/14222 [===========================

2023-05-09 09:09:57.800339: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [29126467,4]
	 [[{{node Placeholder/_1}}]]
2023/05/09 09:10:03 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


The time taken to train the model is 0:08:10.330376
  18/4445 [..............................] - ETA: 13s - loss: 0.7023 - accuracy: 0.7053 

2023-05-09 09:10:03.163073: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [9102021,4]
	 [[{{node Placeholder/_1}}]]


4445/4445 [==============================] - 12s 3ms/step - loss: 0.6932 - accuracy: 0.7108
________________________________________
